In [69]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [70]:
df = pd.read_csv('../../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [71]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [72]:
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [73]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [74]:
df['SeniorCitizen'] = df['SeniorCitizen'].map({0: 'no', 1: 'yes'})


In [75]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [76]:
df.churn = (df.churn == 'yes').astype(int)

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [79]:
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

In [80]:
y_train = df_train.churn.values
y_val = df_val.churn.values

In [81]:
del df_train['churn']
del df_val['churn']

In [82]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [83]:
from sklearn.feature_extraction import DictVectorizer

dv  = DictVectorizer(sparse=False)

X_train_vec = df_train[categorical + numerical].to_dict(orient = 'records')

dv.fit(X_train_vec)

X_train = dv.transform(X_train_vec)

In [84]:
dv = DictVectorizer(sparse=False)

X_val_vec = df_val[categorical + numerical].to_dict(orient = 'records')

dv.fit(X_val_vec)

X_val = dv.transform(X_val_vec)

In [85]:
X_val.shape

(1860, 46)

In [86]:
dv.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'dependents=no', 'dependents=yes',
       'deviceprotection=no', 'deviceprotection=no_internet_service',
       'deviceprotection=yes', 'gender=female', 'gender=male',
       'internetservice=dsl', 'internetservice=fiber_optic',
       'internetservice=no', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'onlinebackup=no', 'onlinebackup=no_internet_service',
       'onlinebackup=yes', 'onlinesecurity=no',
       'onlinesecurity=no_internet_service', 'onlinesecurity=yes',
       'paperlessbilling=no', 'paperlessbilling=yes', 'partner=no',
       'partner=yes', 'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'seniorcitizen=no',
       'seniorcitizen=yes', 'streamingmovies=no',
       'stream

In [87]:
# Now Apply The SMOTE
from imblearn.over_sampling import SMOTE
oversample = SMOTE()

X_train_smote, y_train_smote = oversample.fit_resample(X_train, y_train)
X_val_smote, y_val_smote = oversample.fit_resample(X_val, y_val)

In [88]:
import pickle
import mlflow
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Telecom Churn Prediction 2")

2023/07/09 14:16:46 INFO mlflow.tracking.fluent: Experiment with name 'Telecom Churn Prediction 2' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///c:/Users/micheal.getachew/OneDrive - Safaricom '
 'Ethiopia/Desktop/Projects/Churn_Prediction_Model/notebooks/tracked/mlruns/2'), creation_time=1688937406507, experiment_id='2', last_update_time=1688937406507, lifecycle_stage='active', name='Telecom Churn Prediction 2', tags={}>

In [89]:
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, STATUS_OK
from hyperopt.pyll import scope

In [90]:
import xgboost as xgb
train = xgb.DMatrix(X_train_smote, label=y_train_smote)
valid = xgb.DMatrix(X_val_smote, label=y_val_smote)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        clf = XGBClassifier(**params)
        score = cross_val_score(clf, X_train, y_train, cv=5).mean()
        mlflow.log_metric("score", score)
    return {'loss': score, 'status': STATUS_OK}

space = {
    'max_depth': hp.choice('max_depth', range(1, 20)),
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'n_estimators': hp.choice('n_estimators', range(1, 1000)),
    'gamma': hp.loguniform('gamma', -5, 0),
    'min_child_weight': hp.choice('min_child_weight', range(1, 10)),
    'subsample': hp.uniform('subsample', 0.1, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),
    'reg_alpha': hp.uniform('reg_alpha', 0.01, 100),
    'reg_lambda': hp.uniform('reg_lambda', 0.01, 100)
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100)

print(best)

# def objective(params):
#     with mlflow.start_run():
#         mlflow.set_tag("model", "xgboost")
#         mlflow.log_params(params)
#         booster = xgb.train(
#             params=params,
#             dtrain=train,
#             num_boost_round=1000,
#             evals=[(valid, 'validation')],
#             early_stopping_rounds=50
#         )
#         y_pred = booster.predict(valid)
#         rmse = mean_squared_error(y_val, y_pred, squared=False)
#         mlflow.log_metric("rmse", rmse)

#     return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp

# Define search space
space = {
    'max_depth': hp.choice('max_depth', range(1, 20)),
    'learning_rate': hp.quniform('learning_rate', 0.01, 0.5, 0.01),
    'n_estimators': hp.choice('n_estimators', range(1, 1000)),
    'gamma': hp.quniform('gamma', 0, 1, 0.01),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    'subsample': hp.quniform('subsample', 0.1, 1, 0.01),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.1, 1.0, 0.01)
}

# Define objective function
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("Model", "Xgboost")
        mlflow.log_params(params)
        clf = xgb.XGBClassifier(**params)
        clf.fit(X_train_smote, y_train_smote)
        score = clf.score(X_val_smote, y_val_smote)
    return {'loss': -score, 'status': STATUS_OK}

# Run hyperparameter search
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100)

# Put this section for latter use and ofcourse it is important

# Train final model with best hyperparameters
# clf = xgb.XGBClassifier(**best)
# clf.fit(X_train, y_train)